In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark locourse")\
    .master("local[*]")\
    .config("spark.driver.memory","10g") \
    .getOrCreate()

[ WARN] Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
surl = 'neo4j://localhost:7692'
spassword = 'password'
#turl='neo4j+s://c7df39d1.databases.neo4j.io'
#tpassword = 'B2CgyPs2DgVpKX223mrSniaBkkzA9KCWOLnBuv7IILw'
#dmart2
turl='neo4j://localhost:7689'
tpassword = 'password'

In [3]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Course)-[HAS_LEVEL]->(l:Level) return c.id as id, c.fee as fee, c.link as link, c.name as name, c.rating as rating, c.time as time, c.timestamp as timestamp, l.level as level") \
  .load()

In [4]:
course_info = df.select(df.id, df.fee, df.link, df.name, df.rating, df.time, df.timestamp)

In [5]:
course_info.write.format("org.neo4j.spark.DataSource")\
    .mode("Append")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "id")\
    .option("labels",":Course")\
    .save()

In [5]:
course_level = df.select(df.level)

In [7]:
course_level.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "level")\
    .option("labels",":Level")\
    .save()

In [6]:
course_level_rel = df.dropDuplicates(['id','level']).select(df.id, df.level)

In [10]:
course_level_rel.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("relationship", 'HAS_LEVEL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Level')\
    .option('relationship.target.node.keys','level:level')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [7]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Course)-[HAVE_LANGUAGE]->(l:Language) return distinct c.id as id, l.language as language") \
  .load()

In [8]:
language = df.dropDuplicates(["language"]).select(df.language)

In [16]:
language.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "language")\
    .option("labels",":Language")\
    .save()

In [9]:
course_language_rel = df.select(df.id, df.language)

In [10]:
course_language_rel.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
  .option('batch.size','2000') \
    .option("relationship", 'HAVE_LANGUAGE')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Course')\
    .option('relationship.source.node.keys','id:id')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Language')\
    .option('relationship.target.node.keys','language:language')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [3]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Course)-[TEACH_BY]->(i:Instructor) return distinct c.id as id, i.instructor as instructor") \
  .load()

In [4]:
instructor = df.dropDuplicates(["instructor"]).select(df.instructor)

In [13]:
instructor.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "instructor")\
    .option("labels",":Instructor")\
    .save()

In [10]:
course_ins_rel = df.select('id', 'instructor')

In [11]:
course_ins_rel.count()

15837

In [12]:
course_ins_rel.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("relationship", 'INSTRUCT')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Instructor')\
    .option('relationship.source.node.keys','instructor:instructor')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Course')\
    .option('relationship.target.node.keys','id:id')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [13]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Course)-[TEACH_PL]->(pl:ProgrammingLanguage) return distinct c.id as id, pl.name as name") \
  .load()

In [14]:
pl = df.dropDuplicates(["name"]).select(df.name)
pl.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "name")\
    .option("labels",":ProgrammingLanguage")\
    .save()

In [15]:
course_pl = df
course_pl.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("relationship", 'TAUGHT_PL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':ProgrammingLanguage')\
    .option('relationship.source.node.keys','name:name')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Course')\
    .option('relationship.target.node.keys','id:id')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [16]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Course)-[TEACH_TOOL]->(t:Tool) return distinct c.id as id, t.name as name") \
  .load()

In [17]:
tool = df.dropDuplicates(["name"]).select(df.name)
tool.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "name")\
    .option("labels",":Tool")\
    .save()

In [18]:
course_tool = df
course_tool.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("relationship", 'TAUGHT_TOOL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Tool')\
    .option('relationship.source.node.keys','name:name')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Course')\
    .option('relationship.target.node.keys','id:id')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [19]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Course)-[TEACH_FRAM]->(f:Framework) return distinct c.id as id, f.name as name") \
  .load()

In [20]:
fram = df.dropDuplicates(["name"]).select(df.name)
fram.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "name")\
    .option("labels",":Framework")\
    .save()

In [21]:
course_fram = df
course_fram.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("relationship", 'TAUGHT_FRAM')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Framework')\
    .option('relationship.source.node.keys','name:name')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Course')\
    .option('relationship.target.node.keys','id:id')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [22]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Course)-[TEACH_PLAT]->(p:Platform) return distinct c.id as id, p.name as name") \
  .load()

In [23]:
plat = df.dropDuplicates(["name"]).select(df.name)
plat.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "name")\
    .option("labels",":Platform")\
    .save()

In [24]:
course_plat = df
course_plat.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("relationship", 'TAUGHT_PLAT')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Platform')\
    .option('relationship.source.node.keys','name:name')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Course')\
    .option('relationship.target.node.keys','id:id')\
    .option('relationship.target.save.mode','Match')\
    .save()

In [25]:
df = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", surl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", spassword)\
  .option("query", "match (c:Course)-[TEACH_KNOW]->(k:Knowledge) return distinct c.id as id, k.name as name") \
  .load()

In [26]:
know = df.dropDuplicates(["name"]).select(df.name)
know.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("node.keys", "name")\
    .option("labels",":Knowledge")\
    .save()

In [27]:
course_know = df
course_know.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
  .option("url", turl)\
  .option("authentication.type", "basic")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", tpassword)\
    .option("relationship", 'TAUGHT_KNOW')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Knowledge')\
    .option('relationship.source.node.keys','name:name')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':Course')\
    .option('relationship.target.node.keys','id:id')\
    .option('relationship.target.save.mode','Match')\
    .save()